Technological Institute of the Philippines | Quezon City - Computer Engineering
--- | ---
Course Code: | CPE 018
Code Title: | Emerging Technologies in CpE 1 - Fundamentals of Computer Vision
1st Semester | AY 2023-2024
<hr> | <hr>
<u>**ACTIVITY NO.** | **TITLE**
**Name** | LastName, FirstName
**Section** | CPE31Sx
**Date Performed**: |
**Date Submitted**: |
**Instructor**: | Dr. Jonathan V. Taylar / Engr. Verlyn V. Nojor / Engr. Roman M. Richard

<hr>

## 1. Objectives

This activity aims to enable students to perform data preparation and face recognition on their own generated dataset.

## 2. Intended Learning Outcomes (ILOs)
After this activity, the students should be able to:
* Utilize data preparation techniques for images.
* Perform Face Recognition using multiple algorithms.
* Evaluate the performance of different algorithms.

## 3. Procedures and Outputs

### Preparing the training data

Now that we have our data, we need to load these sample pictures into our face recognition algorithms. All face recognition algorithms take two parameters in their `train()` method: an array of images and an array of labels. What do these labels represent? They are the IDs of a certain individual/face so that when face recognition is performed, we not only know the person was recognized but also who—among the many people available in our database—the person is.

To do that, we need to create a comma-separated value (CSV) file, which will contain the path to a sample picture followed by the ID of that person.

**Include a Screenshot of Your Dataset Here**

---

### Loading the data and recognizing faces

Next up, we need to load these two resources (the array of images and CSV file) into the face recognition algorithm, so it can be trained to recognize our face. To do this, we build a function that reads the CSV file and—for each line of the file—loads the image at the corresponding path into the images array and the ID into the labels array.

In [119]:
import numpy as np
import os
import cv2

def read_images(path, sz=None):
    if not os.path.exists(path): 
        return None, None  

    c = 0
    X, y = [], []

    for dirname, dirnames, filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                if filename.startswith('.'):  
                    continue

                filepath = os.path.join(subject_path, filename)
                im = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

                if im is None:  
                    continue

                if sz is not None:
                    im = cv2.resize(im, (200, 200))

                X.append(np.asarray(im, dtype=np.uint8))
                y.append(c)

            c += 1  

    return X, y


dataset_path = r"C:\Users\blizz\Downloads\faceset"
X, y = read_images(dataset_path, sz=(200, 200))

if X is not None and y is not None:
    print(f"Loaded {len(X)} images with {len(set(y))} unique labels.")
else:
    print("Dataset not found or empty.")



Loaded 40 images with 2 unique labels.


**Question: Run the function above on your generated dataset. Provide an analysis and note all the challenges you have encountered running this code.**

it doesn't filter out non-image files and doesn't have a label dictionary which also leads to errors, storing all in just numpy array.

---

### Performing Face Recognition Algorithms

Here is a sample script for testing the Face Recognition Algorithm. In this section, we're going to follow the same process but with different algorithms for face recognitions, namely:
- Eigenface Recognition
- Fisherface Recognition
- Local Binary Pattern Histograms (LBPH) Recognition

In [80]:
import numpy as np
import os
import cv2

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Paths
MODEL_PATH = "face_model.yml"
LABEL_DICT_PATH = "label_dictionary.npy"
IMAGE_SIZE = (200, 200)

def face_rec():
    names = np.load(LABEL_DICT_PATH, allow_pickle=True).item()  # Load label dictionary

    # Load the trained model
    model = cv2.face.EigenFaceRecognizer_create()
    model.read(MODEL_PATH)  # Load pre-trained model

    camera = cv2.VideoCapture(0)

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(40, 40))

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi = cv2.resize(gray[y:y + h, x:x + w], IMAGE_SIZE, interpolation=cv2.INTER_LINEAR)

            try:
                label_id, confidence = model.predict(roi)
                label = names.get(label_id, "Unknown")  # Get name from dictionary
                cv2.putText(img, f"{label}, {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            except:
                continue

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()


In [126]:
import numpy as np
import os
import cv2

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Paths
DATASET_PATH = "faceset/"
MODEL_PATH = "face_model.yml"
LABEL_DICT_PATH = "label_dictionary.npy"
IMAGE_SIZE = (200, 200)

def read_images(path):
    """ Reads images, detects faces, and prepares training data. """
    X, y = [], []
    label_dict = {}
    label_id = 0  

    for subdirname in sorted(os.listdir(path)):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path):
            continue  

        label_dict[label_id] = subdirname  

        for filename in sorted(os.listdir(subject_path)):
            filepath = os.path.join(subject_path, filename)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"Skipping {filename}: Invalid image")
                continue  

            # Preprocess image (Histogram Equalization for better contrast)
            img = cv2.equalizeHist(img)

            faces = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=6, minSize=(40, 40))
            if len(faces) == 0:
                print(f"Skipping {filename}: No face detected")
                continue  

            for (x, y_, w, h) in faces:
                face = img[y_:y_ + h, x:x + w]
                face = cv2.resize(face, IMAGE_SIZE)  

                X.append(face)
                y.append(label_id)
                break  

        label_id += 1

    return X, y, label_dict

# Read images and extract faces
print("Reading images and training the model...")
X, y, label_dict = read_images(DATASET_PATH)

if len(X) == 0 or len(y) == 0:
    print("Error: No faces found. Check your dataset.")
    exit()

X = np.array(X, dtype=np.uint8)
y = np.array(y, dtype=np.int32)

# Create and train EigenFace recognizer
recognizer = cv2.face.EigenFaceRecognizer_create()
recognizer.train(X, y)

# Save trained model and labels
recognizer.save(MODEL_PATH)
np.save(LABEL_DICT_PATH, label_dict)

print(f"Training complete! Model saved as '{MODEL_PATH}'. Label dictionary saved as '{LABEL_DICT_PATH}'.")
print("Label dictionary:", label_dict)


Reading images and training the model...
Training complete! Model saved as 'face_model.yml'. Label dictionary saved as 'label_dictionary.npy'.
Label dictionary: {0: 'dad', 1: 'john'}


In [127]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
DATASET_PATH, MODEL_PATH, LABEL_PATH = "faceset/", "eigen_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE = (200, 200)
CONFIDENCE_THRESHOLD = 6000

def read_images(path):
    X, y, labels = [], [], {}
    for label_id, subdir in enumerate(sorted(os.listdir(path))):
        subpath = os.path.join(path, subdir)
        if not os.path.isdir(subpath): continue
        labels[label_id] = subdir  
        for file in sorted(os.listdir(subpath)):
            img = cv2.imread(os.path.join(subpath, file), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            faces = face_cascade.detectMultiScale(cv2.equalizeHist(img), 1.2, 6, minSize=(40, 40))
            for (x, y_, w, h) in faces:
                X.append(cv2.resize(img[y_:y_ + h, x:x + w], IMAGE_SIZE))
                y.append(label_id)
    if not X: raise ValueError("No faces detected.")
    return np.array(X, np.uint8), np.array(y, np.int32), labels

def train_model():
    X, y, labels = read_images(DATASET_PATH)
    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)
    model.save(MODEL_PATH)
    np.save(LABEL_PATH, labels)
    return model, labels

def recognize_faces(model, labels):
    cam = cv2.VideoCapture(0)
    if not cam.isOpened(): return print("Camera error.")
    print("Press 'q' to quit.")
    while True:
        ret, img = cam.read()
        if not ret: break
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, 6, minSize=(50, 50))
        for (x, y, w, h) in faces:
            roi = cv2.resize(gray[y:y + h, x:x + w], IMAGE_SIZE)
            label, conf = model.predict(roi)
            text = labels.get(label, "Unknown") if conf < CONFIDENCE_THRESHOLD else "Unknown"
            cv2.putText(img, f"{text}, {conf:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"): break
    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    try:
        model, labels = train_model()
        recognize_faces(model, labels)
    except Exception as e:
        print("Error:", e)


Press 'q' to quit.


**Question: Provide an analysis of the sample script for the process using the Eigenface Model. What is the sample code doing? Are you able to troubleshoot any problems encountered?**

The script implements an Eigenface-based face recognition system using OpenCV, following three main steps: data preparation, model training, and real-time recognition. It reads images, detects faces using Haar cascade, resizes them, and assigns labels before training an EigenFaceRecognizer model. During real-time recognition, the webcam captures faces, predicts identities, and labels them based on confidence scores.

---
Perform the remaining face recognition techniques by using the same (or modified) process from the sample code:

- `model = cv2.face.createFisherFaceRecognizer()`
- `model = cv2.face.createLBPHFaceRecognizer()`

In [135]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
DATASET_PATH, FISHER_MODEL_PATH, LABEL_DICT_PATH = "faceset/", "fisher_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE = (200, 200)

def read_images(path):
    X, y, label_dict = [], [], {}
    for label_id, subdirname in enumerate(sorted(os.listdir(path))):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path): continue
        label_dict[label_id] = subdirname  
        for filename in sorted(os.listdir(subject_path)):
            img = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            img = cv2.equalizeHist(img)
            faces = face_cascade.detectMultiScale(img, 1.2, 6, minSize=(40, 40))
            if len(faces) == 0: continue
            X.append(cv2.resize(img[faces[0][1]:faces[0][1] + faces[0][3], faces[0][0]:faces[0][0] + faces[0][2]], IMAGE_SIZE))
            y.append(label_id)
    return np.array(X, np.uint8), np.array(y, np.int32), label_dict

print("Reading images and training...")
X, y, label_dict = read_images(DATASET_PATH)
if X.size == 0: exit("Error: No faces found.")

recognizer = cv2.face.FisherFaceRecognizer_create()
recognizer.train(X, y)
recognizer.save(FISHER_MODEL_PATH)
np.save(LABEL_DICT_PATH, label_dict)
print(f"Model saved as '{FISHER_MODEL_PATH}', Labels saved as '{LABEL_DICT_PATH}'", label_dict)


Reading images and training...
Model saved as 'fisher_face_model.yml', Labels saved as 'label_dictionary.npy' {0: 'dad', 1: 'john'}


In [138]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
DATASET_PATH, MODEL_PATH, LABEL_PATH = "faceset/", "fisher_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE = (200, 200)
CONFIDENCE_THRESHOLD = 60

def read_images(path):
    X, y, labels = [], [], {}
    for label_id, subdir in enumerate(sorted(os.listdir(path))):
        subpath = os.path.join(path, subdir)
        if not os.path.isdir(subpath): continue
        labels[label_id] = subdir  
        for file in sorted(os.listdir(subpath)):
            img = cv2.imread(os.path.join(subpath, file), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            faces = face_cascade.detectMultiScale(cv2.equalizeHist(img), 1.2, 6, minSize=(40, 40))
            for (x, y_, w, h) in faces:
                X.append(cv2.resize(img[y_:y_ + h, x:x + w], IMAGE_SIZE))
                y.append(label_id)
    if not X: raise ValueError("No faces detected.")
    return np.array(X, np.uint8), np.array(y, np.int32), labels

def train_model():
    X, y, labels = read_images(DATASET_PATH)
    model = cv2.face.FisherFaceRecognizer_create()
    model.train(X, y)
    model.save(MODEL_PATH)
    np.save(LABEL_PATH, labels)
    return model, labels

def recognize_faces(model, labels):
    cam = cv2.VideoCapture(0)
    if not cam.isOpened(): return print("Camera error.")
    print("Press 'q' to quit.")
    while True:
        ret, img = cam.read()
        if not ret: break
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, 6, minSize=(50, 50))
        for (x, y, w, h) in faces:
            roi = cv2.resize(gray[y:y + h, x:x + w], IMAGE_SIZE)
            label, conf = model.predict(roi)
            text = labels.get(label, "Unknown") if conf < CONFIDENCE_THRESHOLD else "Unknown"
            cv2.putText(img, f"{text}, {conf:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"): break
    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    try:
        model, labels = train_model()
        recognize_faces(model, labels)
    except Exception as e:
        print("Error:", e)

Press 'q' to quit.


In [139]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
DATASET_PATH, LBPH_MODEL_PATH, LABEL_DICT_PATH = "faceset/", "lbph_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE = (200, 200)

def read_images(path):
    X, y, label_dict = [], [], {}
    for label_id, subdirname in enumerate(sorted(os.listdir(path))):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path): continue
        label_dict[label_id] = subdirname  
        for filename in sorted(os.listdir(subject_path)):
            img = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            img = cv2.equalizeHist(img)
            faces = face_cascade.detectMultiScale(img, 1.2, 6, minSize=(40, 40))
            if len(faces) == 0: continue
            X.append(cv2.resize(img[faces[0][1]:faces[0][1] + faces[0][3], faces[0][0]:faces[0][0] + faces[0][2]], IMAGE_SIZE))
            y.append(label_id)
    return np.array(X, np.uint8), np.array(y, np.int32), label_dict

print("Reading images and training...")
X, y, label_dict = read_images(DATASET_PATH)
if X.size == 0: exit("Error: No faces found.")

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(X, y)
recognizer.save(LBPH_MODEL_PATH)
np.save(LABEL_DICT_PATH, label_dict)
print(f"Model saved as '{LBPH_MODEL_PATH}', Labels saved as '{LABEL_DICT_PATH}'", label_dict)


Reading images and training...
Model saved as 'lbph_face_model.yml', Labels saved as 'label_dictionary.npy' {0: 'dad', 1: 'john'}


In [144]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
DATASET_PATH, MODEL_PATH, LABEL_PATH = "faceset/", "lbph_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE = (200, 200)
CONFIDENCE_THRESHOLD = 100

def read_images(path):
    X, y, labels = [], [], {}
    for label_id, subdir in enumerate(sorted(os.listdir(path))):
        subpath = os.path.join(path, subdir)
        if not os.path.isdir(subpath): continue
        labels[label_id] = subdir  
        for file in sorted(os.listdir(subpath)):
            img = cv2.imread(os.path.join(subpath, file), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            faces = face_cascade.detectMultiScale(cv2.equalizeHist(img), 1.2, 6, minSize=(40, 40))
            for (x, y_, w, h) in faces:
                X.append(cv2.resize(img[y_:y_ + h, x:x + w], IMAGE_SIZE))
                y.append(label_id)
    if not X: raise ValueError("No faces detected.")
    return np.array(X, np.uint8), np.array(y, np.int32), labels

def train_model():
    X, y, labels = read_images(DATASET_PATH)
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X, y)
    model.save(MODEL_PATH)
    np.save(LABEL_PATH, labels)
    return model, labels

def recognize_faces(model, labels):
    cam = cv2.VideoCapture(0)
    if not cam.isOpened(): return print("Camera error.")
    print("Press 'q' to quit.")
    while True:
        ret, img = cam.read()
        if not ret: break
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, 6, minSize=(50, 50))
        for (x, y, w, h) in faces:
            roi = cv2.resize(gray[y:y + h, x:x + w], IMAGE_SIZE)
            label, conf = model.predict(roi)
            text = labels.get(label, "Unknown") if conf < CONFIDENCE_THRESHOLD else "Unknown"
            cv2.putText(img, f"{text}, {conf:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"): break
    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    try:
        model, labels = train_model()
        recognize_faces(model, labels)
    except Exception as e:
        print("Error:", e)


Press 'q' to quit.


**Question: The `predict()` method returns a two-element array. Provide your analysis of the two returned values and their important ince this application.**

The predict() method returns a predicted label (identity) and a confidence score, which measures the match quality. In LBPH, a lower confidence score indicates a better match, whereas higher scores suggest uncertainty. 

## 4. Supplementary Activity

Your accomplisment of the tasks below contribute to the achievement of ILO1, ILO2, and ILO3 for this module.

---

Tasks:
1. Create a new dataset for testing, this dataset must include the following:
  - The same person/s that the model has to recognize.
  - Different person/s that the model should not recognize.
2. For each model, perform 20 tests. Document the testing performed and provide observations.
3. Conclude on the performed tests by providing your evaluation of the performance of the models.

In [148]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

DATASET_PATH, MODEL_PATH, LABEL_PATH = "faceset/", "eigen_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE, ALLOWED_PERSON, CONFIDENCE_THRESHOLD = (200, 200), "john", 5000 

def read_images(path):
    X, y, labels = [], [], {}  # Ensure y is a list

    for label_id, subdir in enumerate(sorted(os.listdir(path))):
        subpath = os.path.join(path, subdir)
        if not os.path.isdir(subpath): continue
        labels[label_id] = subdir  

        for file in sorted(os.listdir(subpath)):
            img = cv2.imread(os.path.join(subpath, file), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            faces = face_cascade.detectMultiScale(cv2.equalizeHist(img), 1.2, 6, minSize=(40, 40))
            
            for (x, y_pos, w, h) in faces:  
                X.append(cv2.resize(img[y_pos:y_pos + h, x:x + w], IMAGE_SIZE))
                y.append(label_id) 

    if not X:
        raise ValueError("No faces detected.")

    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32), labels  


def train_model():
    X, y, labels = read_images(DATASET_PATH)
    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)
    model.save(MODEL_PATH)
    np.save(LABEL_PATH, labels)
    return model, labels

def recognize_faces(model, labels):
    cam = cv2.VideoCapture(0)
    if not cam.isOpened(): return

    while True:
        ret, img = cam.read()
        if not ret: break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, 6, minSize=(50, 50))

        for (x, y, w, h) in faces:
            roi = cv2.resize(gray[y:y + h, x:x + w], IMAGE_SIZE)
            label, conf = model.predict(roi)
            name = labels.get(label, "Unknown")

            if name == ALLOWED_PERSON and conf < CONFIDENCE_THRESHOLD:
                box_color, text_color, status = (0, 255, 0), (0, 255, 0), "RECOGNIZE FACE"
            else:
                name, box_color, text_color, status = "Unknown", (0, 0, 255), (0, 0, 255), "UNKNOWN FACE"

            cv2.rectangle(img, (x, y), (x + w, y + h), box_color, 3)
            cv2.putText(img, status, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            cv2.putText(img, "MODEL USED: EIGENFACES", (x, y - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            
            if name != "Unknown":
                cv2.putText(img, f"NAME: {name}", (x + w + 15, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
                cv2.putText(img, f"ID: {label}", (x + w + 15, y + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

            cv2.putText(img, "CONFIDENCE LVL:", (x, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            cv2.putText(img, f"{conf:.2f}", (x + 180, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"): break

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    model, labels = train_model()
    recognize_faces(model, labels)


In [149]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

DATASET_PATH, MODEL_PATH, LABEL_PATH = "faceset/", "lbph_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE, ALLOWED_PERSON, CONFIDENCE_THRESHOLD = (200, 200), "john", 60

def read_images(path):
    X, y, labels = [], [], {}

    for label_id, subdir in enumerate(sorted(os.listdir(path))):
        subpath = os.path.join(path, subdir)
        if not os.path.isdir(subpath): continue
        labels[label_id] = subdir  

        for file in sorted(os.listdir(subpath)):
            img = cv2.imread(os.path.join(subpath, file), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            faces = face_cascade.detectMultiScale(cv2.equalizeHist(img), 1.2, 6, minSize=(40, 40))
            
            for (x, y_pos, w, h) in faces:
                X.append(cv2.resize(img[y_pos:y_pos + h, x:x + w], IMAGE_SIZE))
                y.append(label_id)

    if not X:
        raise ValueError("No faces detected.")

    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32), labels  

def train_model():
    X, y, labels = read_images(DATASET_PATH)
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(X, y)
    model.save(MODEL_PATH)
    np.save(LABEL_PATH, labels)
    return model, labels

def recognize_faces(model, labels):
    cam = cv2.VideoCapture(0)
    if not cam.isOpened(): return

    while True:
        ret, img = cam.read()
        if not ret: break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, 6, minSize=(50, 50))

        for (x, y, w, h) in faces:
            roi = cv2.resize(gray[y:y + h, x:x + w], IMAGE_SIZE)
            label, conf = model.predict(roi)
            name = labels.get(label, "Unknown")

            if name == ALLOWED_PERSON and conf < CONFIDENCE_THRESHOLD:
                box_color, text_color, status = (0, 255, 0), (0, 255, 0), "RECOGNIZE FACE"
            else:
                name, box_color, text_color, status = "Unknown", (0, 0, 255), (0, 0, 255), "UNKNOWN FACE"

            cv2.rectangle(img, (x, y), (x + w, y + h), box_color, 3)
            cv2.putText(img, status, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            cv2.putText(img, "MODEL USED: LBPH", (x, y - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            
            if name != "Unknown":
                cv2.putText(img, f"NAME: {name}", (x + w + 15, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
                cv2.putText(img, f"ID: {label}", (x + w + 15, y + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

            cv2.putText(img, "CONFIDENCE LVL:", (x, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            cv2.putText(img, f"{conf:.2f}", (x + 180, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"): break

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    model, labels = train_model()
    recognize_faces(model, labels)


In [156]:
import numpy as np
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

DATASET_PATH, MODEL_PATH, LABEL_PATH = "faceset/", "fisher_face_model.yml", "label_dictionary.npy"
IMAGE_SIZE, ALLOWED_PERSON, CONFIDENCE_THRESHOLD = (200, 200), "john", 300


def read_images(path):
    X, y, labels = [], [], {}

    for label_id, subdir in enumerate(sorted(os.listdir(path))):
        subpath = os.path.join(path, subdir)
        if not os.path.isdir(subpath): continue
        labels[label_id] = subdir  

        for file in sorted(os.listdir(subpath)):
            img = cv2.imread(os.path.join(subpath, file), cv2.IMREAD_GRAYSCALE)
            if img is None: continue
            faces = face_cascade.detectMultiScale(cv2.equalizeHist(img), 1.2, 6, minSize=(40, 40))
            
            for (x, y_pos, w, h) in faces:
                X.append(cv2.resize(img[y_pos:y_pos + h, x:x + w], IMAGE_SIZE))
                y.append(label_id)

    if not X:
        raise ValueError("No faces detected.")

    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32), labels  

def train_model():
    X, y, labels = read_images(DATASET_PATH)
    model = cv2.face.FisherFaceRecognizer_create()
    model.train(X, y)
    model.save(MODEL_PATH)
    np.save(LABEL_PATH, labels)
    return model, labels

def recognize_faces(model, labels):
    cam = cv2.VideoCapture(0)
    if not cam.isOpened(): return

    while True:
        ret, img = cam.read()
        if not ret: break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, 6, minSize=(50, 50))

        for (x, y, w, h) in faces:
            roi = cv2.resize(gray[y:y + h, x:x + w], IMAGE_SIZE)
            label, conf = model.predict(roi)
            name = labels.get(label, "Unknown")

            if name == ALLOWED_PERSON and conf < CONFIDENCE_THRESHOLD:
                box_color, text_color, status = (0, 255, 0), (0, 255, 0), "RECOGNIZE FACE"
            else:
                name, box_color, text_color, status = "Unknown", (0, 0, 255), (0, 0, 255), "UNKNOWN FACE"

            cv2.rectangle(img, (x, y), (x + w, y + h), box_color, 3)
            cv2.putText(img, status, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            cv2.putText(img, "MODEL USED: FISHERFACES", (x, y - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            
            if name != "Unknown":
                cv2.putText(img, f"NAME: {name}", (x + w + 15, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
                cv2.putText(img, f"ID: {label}", (x + w + 15, y + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

            cv2.putText(img, "CONFIDENCE LVL:", (x, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
            cv2.putText(img, f"{conf:.2f}", (x + 180, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord("q"): break

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    model, labels = train_model()
    recognize_faces(model, labels)

    



## 5. Summary, Conclusions and Lessons Learned

### **Summary**
This exercise entailed the application of face recognition with different algorithms on a dataset created by oneself. It focused on data preparation, training, and evaluation of the model, with special emphasis on how preprocessing affects accuracy.

### **Conclusions**
I effectively implemented face recognition algorithms and saw how various algorithms react to lighting and angle changes. Results highlighted the significance of robust preprocessing and data quality in reducing errors.

### **Lessons Learned**
Effective data preparation is important for accuracy since bad input impacts performance. Various algorithms produce bad results, and environmental conditions such as lighting influence recognition. Analysis of multiple models is important for choosing the optimum approach.

<hr/>

***Proprietary Clause***

*Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated, or reduced to any electronic medium or machine-readable form, in whole or in part, without the prior consent of T.I.P.*